In [1]:
import numpy as np
import pandas as pd
import math
from numpy import linalg as LA

In [2]:
# Problem 1

In [3]:
df = pd.DataFrame([[1, 2], [4, 2], [1, 3], [4, 3]], columns = ['feature_1', 'feature_2'])
# initialize a random matrix (provided in the powerpoint)
random_matrix = np.array([-0.18671, 0.72579])
initialize_array = np.array([1,1])
# k = number of classes
k = 2
# d = number of features
# N = total number of observations per feature
d = 2
N = df.shape[0]


In [4]:
def create_df(df):
    mean_list = []
    std_list = []
    for col in df.columns:
        mean_list.append(df[col].mean())
        std_list.append(df[col].std())
        
    mean_array = np.array(mean_list)
    
    std_array = np.array(std_list)
    
    x_matrix = np.array(df.values)
    
    return mean_array, std_array, x_matrix

In [5]:
def expectation_mean_std_matrix(df, mean_array, std_array, random_matrix, initialize_array):
    left_matrix = np.matmul(mean_array.reshape(-1, 1), initialize_array.reshape(1, -1))
    right_matrix = np.matmul(std_array.reshape(-1, 1), random_matrix.reshape(1, -1))
    mean_matrix = np.add(left_matrix, right_matrix)
    average_std = np.array(std_array.mean())
    std_matrix = np.matmul(average_std.reshape(1, 1), initialize_array.reshape(1, -1))
    return(mean_matrix, std_matrix)

In [6]:
def expectation_prob(k, initialize_array, df, mean_matrix, std_matrix, x_matrix):
    prob_k_n = initialize_array.reshape(1, -1) / k
    
    final_list = []
    for i in range(mean_matrix.shape[1]):
        mean_array_new = np.repeat([mean_matrix[:, i]], x_matrix.shape[0], axis = 0)

        norm_array = np.subtract(x_matrix, mean_array_new)
        
        left = 1/(math.sqrt(2*math.pi)* std_matrix[0][i])**2 * np.exp((-1/2))
        
        output_list = left * LA.norm(norm_array, ord = 2, axis = 1)**2 / std_matrix[0][i] ** 2

        final_list.append(output_list)
    
    final_array = np.array(final_list)
    
    sum_p_k_g = np.matmul(prob_k_n, final_array)
    
    mean_list_output = []

    for i in range(prob_k_n.shape[1]):
        mean_list_output.append(prob_k_n[:, i] * final_array[i, ]/sum_p_k_g)
    
    prob_k_n_matrix = np.array(mean_list_output)
    
    return prob_k_n_matrix
        

In [7]:
def maximization_mean_std_matrix(prob_k_n_matrix, x_matrix, d, N, k):
    sum_matrix = np.sum(prob_k_n_matrix.reshape(prob_k_n_matrix.shape[0], prob_k_n_matrix.shape[-1]), axis = 1)
    final_sum = np.repeat([sum_matrix], x_matrix.shape[-1], axis = 0)
    mean_array = np.matmul(prob_k_n_matrix, x_matrix).transpose().reshape(x_matrix.shape[-1], -1)/final_sum

    output_list = []
    for i in range(k):
        x_m_array = np.repeat([mean_array[:, i]], x_matrix.shape[0], axis = 0)
        norm_array_new = np.subtract(x_matrix, x_m_array)
        left_array = prob_k_n_matrix.reshape(-1, x_matrix.shape[0])[i].reshape(x_matrix.shape[0], 1)
        sigma_array = left_array * (norm_array_new ** 2)
        output_list.append(np.sqrt(np.sum(sigma_array, axis = 1).sum()/ (d * sum_matrix[i])))

    sigma_array_new = np.array([output_list])

    new_prob = (1/N) * sum_matrix.reshape(k, -1)

    return mean_array, sigma_array_new, new_prob

In [8]:
def EM(df, random_matrix, initialize_array, k, d, N):
    mean_array, std_array, x_matrix = create_df(df)
    mean_matrix, std_matrix = expectation_mean_std_matrix(df, mean_array, std_array, random_matrix, initialize_array)
    prob_k_n_matrix = expectation_prob(k, initialize_array, df, mean_matrix, std_matrix, x_matrix)
    
    output_dict = {'iteration_1': [mean_matrix, std_matrix]}
    for i in range(1, 6):
        prob_k_n_matrix = 
        expectation_prob(k, initialize_array, df, output_dict['iteration_' + str(i)][0], 
                         output_dict['iteration_' + str(i)][1], x_matrix)
        output_dict['iteration_' + str(i)].append(prob_k_n_matrix)
        mean_updated_array, std_updated_array, new_prob 
        = maximization_mean_std_matrix(prob_k_n_matrix, x_matrix, d, N, k)
        output_dict.update({'iteration_' + str(i + 1) : [mean_updated_array, std_updated_array]})
        output_dict['iteration_' + str(i + 1)][0] = mean_updated_array
        output_dict['iteration_' + str(i + 1)][1] = std_updated_array
        output_dict['iteration_' + str(i)].append(new_prob)
        
    return output_dict

In [9]:
EM(df, random_matrix, initialize_array, k, d, N)

{'iteration_1': [array([[2.17660879, 3.75710516],
         [2.39220293, 2.91903505]]),
  array([[1.15470054, 1.15470054]]),
  array([[[0.15406216, 0.79379697, 0.1873343 , 0.98256436]],
  
         [[0.84593784, 0.20620303, 0.8126657 , 0.01743564]]]),
  array([[0.52943945],
         [0.47056055]])],
 'iteration_2': [array([[3.51638031, 1.3564451 ],
         [2.55242326, 2.44101728]]),
  array([[0.85563511, 0.77097446]]),
  array([[[0.93153816, 0.04713827, 0.90738517, 0.03772544]],
  
         [[0.06846184, 0.95286173, 0.09261483, 0.96227456]]]),
  array([[0.48094676],
         [0.51905324]])],
 'iteration_3': [array([[1.13233851, 3.76725413],
         [2.49127611, 2.50808342]]),
  array([[0.56099284, 0.6685812 ]]),
  array([[[0.06188972, 0.98203935, 0.06591248, 0.98298768]],
  
         [[0.93811028, 0.01796065, 0.93408752, 0.01701232]]]),
  array([[0.52320731],
         [0.47679269]])],
 'iteration_4': [array([[3.81679987, 1.05501285],
         [2.50118765, 2.49869674]]),
  array([[0.6

In [10]:
# Problem 2

In [11]:
iris_df = pd.read_csv('iris.csv')

In [12]:
iris = iris_df.loc[:, 'Sepal.Length': 'Petal.Width']

In [13]:
initialize_array = np.array([1,1,1])
random_matrix = np.array([-0.18671, 0.72579, 0.52579])
k = 3
d = 4
N = iris.shape[0]

In [14]:
EM(iris, random_matrix, initialize_array, k, d, N)

{'iteration_1': [array([[5.68872511, 6.44433545, 6.27872222],
         [2.97595274, 3.37368072, 3.28650747],
         [3.42840117, 5.0392358 , 4.68617616],
         [1.05701594, 1.75255781, 1.60011028]]),
  array([[0.94786708, 0.94786708, 0.94786708]]),
  array([[[1.47275580e-01, 1.42514865e-01, 1.50953859e-01, 1.44853617e-01,
           1.50344626e-01, 1.44200133e-01, 1.50828848e-01, 1.42302813e-01,
           1.51480714e-01, 1.41241275e-01, 1.47762215e-01, 1.41477420e-01,
           1.45673577e-01, 1.64143494e-01, 1.70865561e-01, 1.74986063e-01,
           1.58861416e-01, 1.45803274e-01, 1.43999421e-01, 1.50051135e-01,
           1.31519521e-01, 1.45784496e-01, 1.67992453e-01, 1.25820374e-01,
           1.30611368e-01, 1.33038682e-01, 1.35083672e-01, 1.42766431e-01,
           1.44685739e-01, 1.40249410e-01, 1.37130926e-01, 1.36908119e-01,
           1.62794682e-01, 1.68803542e-01, 1.39337641e-01, 1.50912824e-01,
           1.50860795e-01, 1.52951767e-01, 1.55028444e-01, 1.41312054e-

In [15]:
iris_df.groupby(['Species'])['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'].mean()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [16]:
# Problem 3

In [17]:
mean = [4.5, 2.2, 3.3]

In [18]:
cov = [[0.5, 0.1, 0.05], [0.1, 0.25, 0.1], [0.05, 0.1, 0.4]]

In [19]:
generated_data = np.random.multivariate_normal(mean, cov, 300)

In [20]:
df_generated = pd.DataFrame(generated_data, columns = ['feature_1', 'feature_2', 'feature_3'])

In [21]:
initialize_array = np.array([1,1,1])
random_matrix = np.array([-0.19671, 0.32579, 0.82579])
k = 3
d = 3
N = df_generated.shape[0]

In [22]:
fea_dict = {'feature_1': [3.5, 5.5], 'feature_2': [1.7, 2.7], 'feature_3': [2.5, 4.1]}

In [23]:
for key, value in fea_dict.items():
    df_generated[key][df_generated[key] < fea_dict[key][0]] = fea_dict[key][0]
    df_generated[key][df_generated[key] > fea_dict[key][0]] = fea_dict[key][1]

In [24]:
EM(df_generated, random_matrix, initialize_array, k, d, N)

{'iteration_1': [array([[5.2330895 , 5.51706458, 5.78881106],
         [2.42501476, 2.63304411, 2.83211525],
         [3.83201547, 4.09051064, 4.33787445]]),
  array([[0.47878762, 0.47878762, 0.47878762]]),
  array([[[0.2258113 , 0.57396407, 0.57396407, 0.2258113 , 0.57396407,
           0.57396407, 0.57396407, 0.22551011, 0.2513416 , 0.57396407,
           0.57396407, 0.2421298 , 0.57396407, 0.57396407, 0.2258113 ,
           0.57396407, 0.57396407, 0.2258113 , 0.57396407, 0.57396407,
           0.57396407, 0.23820958, 0.57396407, 0.57396407, 0.57396407,
           0.57396407, 0.2258113 , 0.2258113 , 0.2258113 , 0.57396407,
           0.57396407, 0.57396407, 0.57396407, 0.57396407, 0.23820958,
           0.57396407, 0.57396407, 0.57396407, 0.2258113 , 0.57396407,
           0.57396407, 0.57396407, 0.57396407, 0.2513416 , 0.57396407,
           0.57396407, 0.23820958, 0.2258113 , 0.57396407, 0.57396407,
           0.23393907, 0.2258113 , 0.57396407, 0.57396407, 0.22551011,
           0